# Initialize Stack for EKS Provosioning
## Create Stack using Cloud Formation (VPC, etc.)

### Required following CLIs installed in your mac.
- aws (<a href="https://formulae.brew.sh/formula/awscli">brew install awscli</a>) 
- jq (<a href="https://formulae.brew.sh/formula/jq">brew install jq</a>)
- <a href="https://github.com/pivotal-cf/pivnet-cli/releases/tag/v3.0.1">pivnet</a>
- kubectl (<a href="">brew install kubernetes-cli</a>)


### Note: make sure to configure aws cli to use your credentials. Also update below environment variables to your requirements.

In [ ]:
export EKS_REGION=us-east-2
export EKS_NODES_SSH_KEY=tanzu

In [ ]:
aws cloudformation create-stack \
  --region ${EKS_REGION} \
  --stack-name eks-vpc-stack \
  --template-url https://amazon-eks.s3.us-west-2.amazonaws.com/cloudformation/2020-10-29/amazon-eks-vpc-private-subnets.yaml

In [ ]:
Check status to see if Stack creation completed.

In [ ]:
aws cloudformation describe-stacks --region us-east-2 --stack-name eks-vpc-stack --output json | jq -r '.Stacks[].StackStatus'

## Create EKS Control Plane Role

In [ ]:
cat <<EOF > cluster-role-trust-policy.json
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "eks.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
}
EOF

aws iam create-role \
  --role-name AWSEKSClusterRole \
  --assume-role-policy-document file://"cluster-role-trust-policy.json"
    
aws iam attach-role-policy \
  --policy-arn arn:aws:iam::aws:policy/AmazonEKSClusterPolicy \
  --role-name AWSEKSClusterRole

## Create EKS Worker Nodes Role

In [ ]:
cat <<EOF > node-role-trust-policy.json
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "ec2.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
}
EOF

aws iam create-role \
  --role-name AWSEKSNodeRole \
  --assume-role-policy-document file://"node-role-trust-policy.json"

aws iam attach-role-policy \
  --policy-arn arn:aws:iam::aws:policy/AmazonEKSWorkerNodePolicy \
  --role-name AWSEKSNodeRole

aws iam attach-role-policy \
  --policy-arn arn:aws:iam::aws:policy/AmazonEC2ContainerRegistryReadOnly \
  --role-name AWSEKSNodeRole

aws iam attach-role-policy \
  --policy-arn arn:aws:iam::aws:policy/AmazonEKS_CNI_Policy \
  --role-name AWSEKSNodeRole

In [ ]:
export EKS_CLUSTER_ROLE=$(aws iam get-role --role-name AWSEKSClusterRole --output json | jq -r .Role.Arn)
export EKS_NODE_ROLE=$(aws iam get-role --role-name AWSEKSNodeRole --output json | jq -r .Role.Arn)
export EKS_VPC_ID=$(aws cloudformation describe-stacks --region us-east-2 --stack-name eks-vpc-stack --output json | jq -r '.Stacks[].Outputs[] | select(.OutputKey=="VpcId").OutputValue')
export EKS_SUBNETS_ID=$(aws cloudformation describe-stacks --region us-east-2 --stack-name eks-vpc-stack --output json | jq -r '.Stacks[].Outputs[] | select(.OutputKey=="SubnetIds").OutputValue')
export EKS_CONTROL_PLANE_SG=$(aws cloudformation describe-stacks --region us-east-2 --stack-name eks-vpc-stack --output json | jq -r '.Stacks[].Outputs[] | select(.OutputKey=="SecurityGroups").OutputValue')

# Create Control Plan for EKS

### update environment variable below for your cluster name

In [ ]:
export EKS_CLUSTER_NAME=eks-democluster-01
env | grep EKS_

In [ ]:
aws eks create-cluster \
--region ${EKS_REGION} \
--name ${EKS_CLUSTER_NAME} \
--kubernetes-version 1.21 \
--role-arn ${EKS_CLUSTER_ROLE} \
--resources-vpc-config subnetIds=${EKS_SUBNETS_ID},securityGroupIds=${EKS_CONTROL_PLANE_SG}

### Check Status of the EKS ControlPlane ... wait for the controlplane to be Active

In [ ]:
aws eks describe-cluster --name ${EKS_CLUSTER_NAME} | jq -r .cluster.status

# Create Worker Nodes for EKS

In [ ]:
aws eks create-nodegroup \
   --cluster-name ${EKS_CLUSTER_NAME} \
   --region ${EKS_REGION} \
   --nodegroup-name large-vms \
   --disk-size 50 \
   --ami-type AL2_x86_64 \
   --capacity-type ON_DEMAND \
   --instance-types m5.large \
   --node-role ${EKS_NODE_ROLE} \
   --remote-access ec2SshKey=${EKS_NODES_SSH_KEY},sourceSecurityGroups=sg-0ac5b651eb2dc759f \
   --scaling-config desiredSize=5,maxSize=5,minSize=5 \
   --subnets $(echo ${EKS_SUBNETS_ID} | sed "s/,/ /g")

# Access EKS Cluster

In [ ]:
aws eks update-kubeconfig --region ${EKS_REGION} --name ${EKS_CLUSTER_NAME}

In [ ]:
kubectl get nodes